In [1]:
from cassiopeia.TreeSolver import utilities

import pandas as pd 
import numpy as np
import networkx as nx
import pickle as pic

import sys

### Preprocess

https://github.com/YosefLab/Cassiopeia/blob/master/notebooks/reconstruct_lineages.ipynb

In [4]:
alleletable_fp = "GSE146712_IVLT-2B_ALL.alleleTable.txt"
target_lg = 4
alleletable = pd.read_csv(alleletable_fp, sep='\t')

In [5]:
allele_props = utilities.get_indel_props(alleletable, group_var=['intBC', 'lineageGrp'])

Counting unique alleles: 100%|██████████| 117/117 [00:00<00:00, 1431.58it/s]


In [6]:
lg = alleletable[alleletable["lineageGrp"] == target_lg]
cm_fp = "lg4_character_matrix.txt" # relative file path to the cm

utilities.alleletable_to_character_matrix(lg, cm_fp, mutation_map=allele_props, write=True)


Processing characters: 100%|██████████| 27/27 [00:00<00:00, 137.50it/s]

Dropping the following intBCs due to lack of diversity with threshold 1.0: []


### Run reconstruct-lineage Command
reconstruct-lineage --greedy --verbose lg4_character_matrix.txt output.pkl

### Post-Process

In [7]:
g = nx.read_gpickle("output.pkl")
cm = pd.read_csv("lg4_character_matrix.txt", sep='\t', index_col = 0) # read in the associated character matrix

In [8]:
g = g.post_process(cm = cm) # Cassiopeia-Tree's have built in functionality to post-process, callable as a method on the object 
# create a newick string for the given post-processed tree. Note that you need to pass in the 
# networkx object for this

g.newick = utilities.convert_network_to_newick_format(g.network) 

In [9]:
print(g.newick)

with open('test.nwk', 'w') as f:
    f.write(g.newick)

(((((-|-|-|0|5|0|0|32|19|0|0|0|0|4|0|0|0|0|3|3|0|-|-|-|0|0|0|4|0|0|0|4|0|-|-|-|0|0|4|0|5|0|0|4|0|0|0|0|-|-|-,(-|-|-|0|39|0|0|0|0|0|0|0|0|43|38|0|0|3|0|0|26|0|0|0|0|0|0|0|0|0|0|18|8|0|0|0|4|0|0|0|0|3|-|-|-|0|222|0|-|-|-,0|49|19|0|0|0|49|93|49|0|40|13|0|43|38|0|0|0|0|0|0|0|0|0|0|0|16|13|0|0|0|18|8|-|-|-|0|65|0|0|0|0|-|-|-|-|-|-|-|-|-)0|49|19|0|0|0|0|0|0|0|0|0|0|43|38|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|18|8|0|0|0|0|0|0|0|0|0|-|-|-|0|222|0|-|-|-)0|49|19|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|4|0|0|0|0|-|-|-,(-|-|-|0|10|34|5|7|4|0|25|12|50|26|0|0|61|0|0|4|37|-|-|-|0|0|0|0|0|8|0|0|0|-|-|-|42|73|0|0|8|4|0|148|0|24|34|23|-|-|-,-|-|-|0|0|34|5|7|4|0|25|12|50|26|0|0|61|0|0|4|37|-|-|-|3|0|0|32|62|0|0|0|0|-|-|-|42|73|0|0|8|4|0|148|0|24|34|23|-|-|-)-|-|-|0|0|34|5|7|4|0|25|12|50|26|0|0|61|0|0|4|37|-|-|-|0|0|0|0|0|0|0|0|0|-|-|-|42|73|0|0|8|4|0|148|0|24|34|23|-|-|-)0|49|19|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|-|-

In [10]:
## now save final tree
out_fp = "test_lg4_tree.processed.pkl"

# pic.dump(g, open(out_fp, 'wb'))

with open('test_lg4_tree.processed.nwk', 'w') as f:
    f.write(out_fp)